In [133]:
import code_for_hw12 as hw12
import numpy as np
import scipy.spatial.distance as dist

In [216]:
data = hw12.load_ratings_data()
movies_dict, genres_dict = hw12.load_movies()
model = hw12.mf_als(data, None, k=10, lam=1, max_iter=20)
hw12.save_model(model)

Loading from ratings.csv users 13366 items 2000
Iteration 1 finished. Total Elapsed Time: 5.66
Iteration 2 finished. Total Elapsed Time: 11.27
Iteration 3 finished. Total Elapsed Time: 16.86
Iteration 4 finished. Total Elapsed Time: 22.56
Iteration 5 finished. Total Elapsed Time: 28.16
Iteration 6 finished. Total Elapsed Time: 34.15
Iteration 7 finished. Total Elapsed Time: 39.78
Iteration 8 finished. Total Elapsed Time: 45.37
Iteration 9 finished. Total Elapsed Time: 51.11
Iteration 10 finished. Total Elapsed Time: 56.70
Iteration 11 finished. Total Elapsed Time: 62.27
Iteration 12 finished. Total Elapsed Time: 67.84
Iteration 13 finished. Total Elapsed Time: 73.43
Iteration 14 finished. Total Elapsed Time: 79.00
Iteration 15 finished. Total Elapsed Time: 84.75
Iteration 16 finished. Total Elapsed Time: 90.34
Iteration 17 finished. Total Elapsed Time: 95.92
Iteration 18 finished. Total Elapsed Time: 101.52
Iteration 19 finished. Total Elapsed Time: 108.54
Iteration 20 finished. Total 

In [217]:
data = hw12.load_ratings_data()
movies_dict, genres_dict = hw12.load_movies()

Loading from ratings.csv users 13366 items 2000


In [218]:
users_favorite_movies = [data[i][1] for i in range(len(data)) 
                         if data[i][0] == 270894 and data[i][2] == 5.0]

In [219]:
users_favorite_genres_raw = [genres_dict[i] for i in users_favorite_movies]

In [220]:
users_favorite_genres = {}

for genres in users_favorite_genres_raw:
    
    for genre in genres:
        
        if genre not in users_favorite_genres:
            users_favorite_genres[genre] = 0
        
        users_favorite_genres[genre] += 1    

In [221]:
users_favorite_genres

{'Animation': 74,
 'Children': 58,
 'Comedy': 42,
 'Crime': 6,
 'Adventure': 46,
 'Fantasy': 31,
 'Musical': 23,
 'Sci-Fi': 15,
 'IMAX': 12,
 'Drama': 27,
 'Romance': 16,
 'Action': 20,
 'War': 2,
 'Mystery': 4,
 'Thriller': 8,
 'Horror': 3}

In [222]:
model = hw12.load_model()

In [223]:
seen_movies = {data[i][1]: 0 for i in range(len(data)) if data[i][0] == 270894}

In [224]:
predicted_ratings = [(i, np.asscalar(hw12.pred((270894, i, 0), model))) for i in movies_dict if i not in seen_movies]
predicted_ratings = reversed(sorted(predicted_ratings, key=lambda x: x[1]))
predicted_ratings = list(predicted_ratings)

In [225]:
top_50_predicted_movies = [predicted_ratings[i][0] for i in range(50)]

In [226]:
top_predicted_genres_raw = [genres_dict[i] for i in top_50_predicted_movies]

In [227]:
top_predicted_genres = {}

for genres in top_predicted_genres_raw:
    
    for genre in genres:
        
        if genre not in top_predicted_genres:
            top_predicted_genres[genre] = 0
        
        top_predicted_genres[genre] += 1    

In [228]:
top_predicted_genres

{'Adventure': 30,
 'Animation': 25,
 'Children': 21,
 'Comedy': 22,
 'Fantasy': 16,
 'IMAX': 1,
 'Musical': 14,
 'Action': 10,
 'Sci-Fi': 12,
 'Drama': 8,
 'Crime': 1,
 'Thriller': 2,
 'Romance': 5,
 'War': 1,
 'Documentary': 1,
 'Horror': 2}

In [229]:
def similarity(vA, vB):
    
    return (vA.T @ vB) / (np.linalg.norm(vA) * np.linalg.norm(vB))

In [230]:
(u, b_u, v, b_v) = model

similarity_260 = [(i, similarity(v[260], v[i])) for i in movies_dict]
similarity_260 = list(reversed(sorted(similarity_260, key=lambda x: np.abs(x[1]))))

In [231]:
[movies_dict[similarity_260[i][0]] for i in range(0, 11)]

['Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Indiana Jones and the Last Crusade (1989)',
 'Star Wars: Episode VII - The Force Awakens (2015)',
 'Another Stakeout (1993)',
 'Batman (1989)',
 'Threesome (1994)',
 '"Juror, The (1996)"',
 'Battlestar Galactica (2003)']

In [232]:
(u, b_u, v, b_v) = model

similarity_2628 = [(i, similarity(v[2628], v[i])) for i in movies_dict]
similarity_2628 = list(reversed(sorted(similarity_2628, key=lambda x: np.abs(x[1]))))

In [236]:
[movies_dict[similarity_2628[i][0]] for i in range(0, 11)]

['Star Wars: Episode I - The Phantom Menace (1999)',
 'Star Wars: Episode III - Revenge of the Sith (2005)',
 'Star Wars: Episode II - Attack of the Clones (2002)',
 '"Matrix Reloaded, The (2003)"',
 'Indiana Jones and the Kingdom of the Crystal Skull (2008)',
 'State and Main (2000)',
 '"Hard Day\'s Night, A (1964)"',
 '"Matrix Revolutions, The (2003)"',
 'Wag the Dog (1997)',
 '"Dark Knight Rises, The (2012)"',
 '"Ref, The (1994)"']

In [203]:
similarity(v[2628], v[33493])

array([[0.98213317]])

In [239]:
np.mean([similarity(v[i], v[j]) for i in movies_dict for j in movies_dict if i != j])

0.0033906829235349763

In [242]:
genre_similarities = []

for genre in hw12.genres:
    
    movies_same_genre = []
    
    for movie, genres in genres_dict.items():
    
        if genre in genres:
            movies_same_genre.append(movie)
    
    mean = np.mean([similarity(v[i], v[j]) for i in movies_same_genre for j in movies_same_genre if i != j])
    genre_similarities.append((genre, mean))

In [244]:
sorted(genre_similarities, key=lambda x: x[1])

[('Crime', 0.0478018985278635),
 ('Drama', 0.05640458822504264),
 ('Mystery', 0.061131866728382014),
 ('Thriller', 0.07558245350611818),
 ('Adventure', 0.07602538522957676),
 ('Comedy', 0.07945292957559967),
 ('Romance', 0.08449906364727004),
 ('Fantasy', 0.09522018778285715),
 ('War', 0.10288046810178295),
 ('Action', 0.11844986903190609),
 ('Western', 0.12552996595815366),
 ('Sci-Fi', 0.14433745039290163),
 ('Horror', 0.23770689822654634),
 ('Musical', 0.2573167530286519),
 ('Children', 0.28099931396246525),
 ('IMAX', 0.31228193242362234),
 ('Animation', 0.36053878623523145),
 ('Documentary', 0.4194852334967673),
 ('Film-Noir', 0.4215194836150668)]